In [93]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import datetime as dt
from sklearn.metrics import f1_score

In [94]:
smpl_sub = pd.read_csv('sample_submission.csv',index_col='ID')
item = pd.read_csv('items.csv')
s_train = pd.read_csv('sales_train.csv')
s_train['item_cnt_day'] = s_train['item_cnt_day'].astype('int')
# s_train['date'] = pd.to_datetime(s_train['date']).map(dt.datetime.toordinal)
# s_train = s_train.drop(['date','date_block_num'],axis = 1)

In [95]:
# dt.datetime.toordinal(pd.to_datetime(s_train['date']))
# s_train.loc[2,'date']
# s_train[(s_train['item_cnt_day']==2.0)]

In [96]:
test = pd.read_csv('test.csv')
test = test.set_index('ID')
it_cat = pd.read_csv('item_categories.csv')
shop = pd.read_csv('shops.csv')
s_train = s_train.drop(['date'],axis = 1)
s_train.head(3)

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,59,22154,999.0,1
1,0,25,2552,899.0,1
2,0,25,2552,899.0,-1


In [97]:
item.head(3)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40


In [98]:
it_cat.head(3)

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2


In [123]:
cd = test.copy()
cd['item_cnt_month'] = -1000.9
test.head(3)

,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233


In [124]:
# test[(test['item_id']==5320)]
# len(cd)
# ef = s_train[['shop_id','item_id','date_block_num','item_price']]
# ef_grp = ef.groupby(['shop_id','item_id'])
# cd

In [125]:
shop.head(3)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2


In [126]:
smpl_sub.head(3)

,item_cnt_month
ID,
0,0.5
1,0.5
2,0.5


In [127]:
# s_train[(s_train['item_cnt_day']>5)]

In [128]:
# sgroup = s_train.groupby(['shop_id','item_id'])
# tgroup = test.groupby('shop_id')

In [129]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelBinarizer
# type(sgroup.get_group(5))
# tgroup.get_group(1)
# y = LabelBinarizer(ab.item_name)
# print(y)
# ab.corr()
sgroup = s_train.groupby('item_id')
tgroup = test.groupby('shop_id')

In [130]:
# sgroup.get_group((5,5320))
n = sgroup.groups.keys()

In [131]:
# for i in range(len(cd)):
#     a = test.loc[i,'shop_id']
#     b = test.loc[i,'item_id']
#     group = sgroup.get_group((a,b))
#     if (a in test.shop_id.unique())&(b in tgroup.get_group(a).item_id):
#         if len(group)>2:
#             neigh = KNeighborsClassifier(n_neighbors = 2)
#             X = preprocessing.StandardScaler().fit_transform(group[['date_block_num','item_price']].values)
#             X_train, X_test, y_train, y_test = train_test_split( X, group['item_cnt_day'].values, test_size=0, shuffle = False)
#             neigh.fit(X_train,y_train)
#             v = neigh.predict([[ef_grp.get_group((a,b))['date_block_num'].max()+1,ef_grp.get_group((a,b))['item_price'].mean()]])
#             cd.loc[i,'item_cnt_month'] = v
# #           print(f1_score(y_test, neigh.predict(X_test)))
# #         elif 
# #         else:
# #           print(f1_score(y_test, group2['item_cnt_day']))

In [ ]:
for i in range(len(test)):
    a = test.loc[i,'shop_id']
    b = test.loc[i,'item_id']
    if b in n:
        group = sgroup.get_group(b)
        if len(group)>=4:
            neigh = KNeighborsClassifier(n_neighbors = 4,weights = 'distance')
            X_train = group[['date_block_num','shop_id','item_price']].values
#             X_train = preprocessing.StandardScaler().fit_transform(group[['shop_id','item_price']].values)
#             X_train, X_test, y_train, y_test = train_test_split( X, group['item_cnt_day'].values, test_size=0, shuffle = False)
            y_train = group['item_cnt_day'].values
            neigh.fit(X_train,y_train)
            if a in group['shop_id'].unique():
                v = neigh.predict([[34, a, group[group['shop_id']==a]['item_price'].mean()]])
                cd.loc[i,'item_cnt_month'] = v
                print(v," first ")
            else:
                v = np.round(neigh.predict([[34, a, group['item_price'].mean()]]))
                cd.loc[i,'item_cnt_month'] = v
                print(v," second ")
        else:
            cd.loc[i,'item_cnt_month'] = group['item_cnt_day'].mean()
            print(group['item_cnt_day'].mean()," third ")
#             print(group['item_cnt_day'].mean())
    else:
#         v = s_train[s_train['shop_id']==a]['item_cnt_day'].mean()
        cd.loc[i,'item_cnt_month'] = 0
        print(0," fourth ")

[1]  first 
0  fourth 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[3]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[4]  first 
[1]  first 
[1]  first 
[1]  first 
[5]  first 
[3]  first 
[5]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[2]  first 
[1]  first 
[1]  first 
0  fourth 
0  fourth 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  fi

[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
0  fourth 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  second 
0  fourth 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  second 
0  fourth 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[6]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  f

[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[2]  second 
[8]  second 
[1]  first 
[4]  first 
0  fourth 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
1.0  third 
[1]  first 
0  fourth 
[1]  first 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
0  fourth 
[1]  second 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  fir

[1]  second 
[1]  second 
[1]  second 
[1]  second 
1.0  third 
[1]  second 
[1]  first 
[1]  first 
0  fourth 
[1]  second 
1.0  third 
[1]  second 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
[1]  second 
0  fourth 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  second 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
1.0  third 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1] 

[1]  first 
[1]  first 
[1]  first 
[6]  first 
[1]  second 
[1]  second 
[1]  first 
1.0  third 
1.0  third 
0  fourth 
1.0  third 
[1]  second 
[2]  first 
[1]  second 
[2]  second 
[2]  second 
[3]  first 
[1]  second 
[1]  second 
[1]  second 
[2]  second 
[1]  second 
[1]  second 
[1]  second 
[4]  second 
1.3333333333333333  third 
[2]  second 
0  fourth 
[1]  first 
0  fourth 
[1]  second 
[1]  second 
[1]  second 
[2]  second 
0  fourth 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  second 
[3]  second 
[1]  second 
1.0  third 
[1]  first 
2.0  third 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
0  fourth 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  second 
1.0  third 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
1.0  third 
[1]  first 
[1]  first 
[1]  first 
[1]  fi

...

[1]  second 
[1]  second 
[1]  second 
0  fourth 
[1]  second 
[1]  second 
[1]  second 
1.0  third 
[1]  second 
1.0  third 
1.0  third 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[2]  second 
[1]  second 
1.0  third 
[1]  second 
[1]  second 
1.0  third 
[1]  second 
1.0  third 
[1]  second 
[1]  second 
1.0  third 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
0  fourth 
0  fourth 
1.0  third 
0  fourth 
0  fourth 
1.0  third 
0  fourth 
[1]  second 
[1]  second 
0  fourth 
1.0  third 
[1]  second 
[1]  second 
[1]  second 
0  fourth 
[1]  second 
[1]  second 
0  fourth 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
0  fourth 
0  fourth 
1.0  third 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  s

[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[2]  second 
[1]  first 
[1]  first 
[1]  first 
[2]  first 
[1]  second 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[3]  first 
[3]  second 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
0  fourth 
0  fourth 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[6]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  f

[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[3]  first 
[1]  first 
0  fourth 
0  fourth 
0  fourth 
[1]  first 
0  fourth 
[1]  second 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[2]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[2]  first 
[5]  second 
[2]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[4]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
1.0  third 
[1]  first 
1.0  third 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 


[1]  first 
[1]  first 
[2]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
1.0  third 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[6]  first 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[2]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
1.0  third 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
1.5  third 
[2]  second 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1] 

[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  second 
[2]  second 
[1]  second 
[1]  second 
[1]  first 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[1]  second 
[2]  second 
[1]  second 
[2]  second 
[1]  second 
1.0  third 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[3]  first 
[1]  first 
[1]  second 
[1]  first 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
0  fourth 
[1]  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  second 
[1]  second 
[1]  second 
[1]  first 
0  fourth 
[1]  second 
[1]  first

In [ ]:
de = cd[['item_cnt_month']]

In [ ]:
de['item_cnt_month'] = round(de['item_cnt_month'])
# de

In [ ]:
de.to_csv('submssn3.csv')

In [138]:
# de[de['item_cnt_month']==0]

In [ ]:
_